# Local Modeling

## Libraries to Import

In [1]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

from PIL import Image
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras import models, layers, optimizers
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import BatchNormalization

import os
import gc
from timeit import default_timer as timer

## Define Image Directories

In [2]:
train_dir = '/Users/arencarpenter/Desktop/Detecting_ALL_with_CNN/Data/training/'
test_dir = '/Users/arencarpenter/Desktop/Detecting_ALL_with_CNN/Data/testing/'
val_dir = '/Users/arencarpenter/Desktop/Detecting_ALL_with_CNN/Data/validation/'

Create flow_from_directory objects for Keras model. Using a downsized dataset with class balance for local model running.

In [3]:
train_g = image.ImageDataGenerator(rescale = 1/255).flow_from_directory(train_dir,
                                                                  target_size = (256,256), 
                                                                  color_mode='rgb',
                                                                  class_mode='binary')
val_g = image.ImageDataGenerator(rescale = 1/255).flow_from_directory(val_dir,
                                                                target_size = (256,256), 
                                                                color_mode='rgb',
                                                                class_mode='binary')
test_g = image.ImageDataGenerator(rescale = 1/255).flow_from_directory(test_dir,
                                                                target_size = (256,256), 
                                                                color_mode='rgb',
                                                                class_mode='binary')

Found 1346 images belonging to 2 classes.
Found 593 images belonging to 2 classes.
Found 947 images belonging to 2 classes.


## Vanilla Model - 1C1D

In [4]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [5]:
# compiling models
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy', 'Recall'])

In [6]:
# training
start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 410s 10s/step - loss: 1.0253 - accuracy: 0.6478 - recall: 0.6419 - val_loss: 0.6186 - val_accuracy: 0.7386 - val_recall: 0.5612
Epoch 2/5
43/43 [==============================] - 378s 9s/step - loss: 0.6211 - accuracy: 0.7259 - recall: 0.7073 - val_loss: 0.5100 - val_accuracy: 0.7639 - val_recall: 0.8265
Epoch 3/5
43/43 [==============================] - 308s 7s/step - loss: 0.4892 - accuracy: 0.7749 - recall: 0.7816 - val_loss: 0.6214 - val_accuracy: 0.7150 - val_recall: 0.8980
Epoch 4/5
43/43 [==============================] - 306s 7s/step - loss: 0.4327 - accuracy: 0.8061 - recall: 0.8009 - val_loss: 0.7740 - val_accuracy: 0.6358 - val_recall: 0.2721
Epoch 5/5
43/43 [==============================] - 346s 8s/step - loss: 0.4165 - accuracy: 0.8068 - recall: 0.7875 - val_loss: 0.5517 - val_accuracy: 0.7808 - val_recall: 0.7007
Total Time Elapsed:  29  minutes  56  seconds


In [7]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 14s 474ms/step - loss: 1.2477 - accuracy: 0.5164 - recall: 0.3272


## 2C1D

Add another layer.

In [4]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 100s 2s/step - loss: 0.5483 - accuracy: 0.7385 - recall: 0.6909 - val_loss: 0.5160 - val_accuracy: 0.7622 - val_recall: 0.7823
Epoch 2/5
43/43 [==============================] - 103s 2s/step - loss: 0.5017 - accuracy: 0.7697 - recall: 0.7311 - val_loss: 0.5093 - val_accuracy: 0.7656 - val_recall: 0.8061
Epoch 3/5
43/43 [==============================] - 100s 2s/step - loss: 0.4607 - accuracy: 0.7838 - recall: 0.7340 - val_loss: 0.4995 - val_accuracy: 0.7707 - val_recall: 0.7823
Epoch 4/5
43/43 [==============================] - 86s 2s/step - loss: 0.4479 - accuracy: 0.7987 - recall: 0.7756 - val_loss: 0.5273 - val_accuracy: 0.7589 - val_recall: 0.8435
Epoch 5/5
43/43 [==============================] - 92s 2s/step - loss: 0.4196 - accuracy: 0.8039 - recall: 0.7860 - val_loss: 0.5227 - val_accuracy: 0.7673 - val_recall: 0.8265
Total Time Elapsed:  8  minutes  15  seconds


In [5]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 12s 393ms/step - loss: 0.8072 - accuracy: 0.5744 - recall: 0.4676


## 3C1D

Add another layer.

In [6]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 76s 2s/step - loss: 0.5603 - accuracy: 0.7229 - recall: 0.7103 - val_loss: 0.5338 - val_accuracy: 0.7487 - val_recall: 0.8673
Epoch 2/5
43/43 [==============================] - 79s 2s/step - loss: 0.5138 - accuracy: 0.7593 - recall: 0.7207 - val_loss: 0.4887 - val_accuracy: 0.7808 - val_recall: 0.7585
Epoch 3/5
43/43 [==============================] - 78s 2s/step - loss: 0.4982 - accuracy: 0.7727 - recall: 0.7028 - val_loss: 0.5109 - val_accuracy: 0.7572 - val_recall: 0.8333
Epoch 4/5
43/43 [==============================] - 78s 2s/step - loss: 0.4798 - accuracy: 0.7719 - recall: 0.7251 - val_loss: 0.4772 - val_accuracy: 0.7875 - val_recall: 0.7755
Epoch 5/5
43/43 [==============================] - 78s 2s/step - loss: 0.4652 - accuracy: 0.7912 - recall: 0.7623 - val_loss: 0.4864 - val_accuracy: 0.7791 - val_recall: 0.7211
Total Time Elapsed:  6  minutes  41  seconds


In [7]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 14s 475ms/step - loss: 0.9313 - accuracy: 0.4879 - recall: 0.2855


## 2x2x1C1D

Try deeper structure.

In [31]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 231s 5s/step - loss: 0.5679 - accuracy: 0.7266 - recall: 0.6805 - val_loss: 0.5154 - val_accuracy: 0.7589 - val_recall: 0.7483
Epoch 2/5
43/43 [==============================] - 234s 5s/step - loss: 0.5223 - accuracy: 0.7660 - recall: 0.7355 - val_loss: 0.5715 - val_accuracy: 0.7487 - val_recall: 0.8980
Epoch 3/5
43/43 [==============================] - 231s 5s/step - loss: 0.5076 - accuracy: 0.7637 - recall: 0.7340 - val_loss: 0.5879 - val_accuracy: 0.7470 - val_recall: 0.9048
Epoch 4/5
43/43 [==============================] - 230s 5s/step - loss: 0.4998 - accuracy: 0.7734 - recall: 0.7340 - val_loss: 0.5016 - val_accuracy: 0.7589 - val_recall: 0.7925
Epoch 5/5
43/43 [==============================] - 228s 5s/step - loss: 0.4902 - accuracy: 0.7682 - recall: 0.7325 - val_loss: 0.4967 - val_accuracy: 0.7791 - val_recall: 0.7551
Total Time Elapsed:  19  minutes  45  seconds


## 2x2x1C2D

Add Dense layer.

In [32]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 224s 5s/step - loss: 0.5736 - accuracy: 0.7147 - recall: 0.6508 - val_loss: 0.5582 - val_accuracy: 0.7454 - val_recall: 0.8946
Epoch 2/5
43/43 [==============================] - 224s 5s/step - loss: 0.5145 - accuracy: 0.7756 - recall: 0.7415 - val_loss: 0.4981 - val_accuracy: 0.7774 - val_recall: 0.7687
Epoch 3/5
43/43 [==============================] - 225s 5s/step - loss: 0.5088 - accuracy: 0.7682 - recall: 0.7251 - val_loss: 0.5755 - val_accuracy: 0.7386 - val_recall: 0.8912
Epoch 4/5
43/43 [==============================] - 228s 5s/step - loss: 0.4976 - accuracy: 0.7689 - recall: 0.7221 - val_loss: 0.5018 - val_accuracy: 0.7757 - val_recall: 0.7619
Epoch 5/5
43/43 [==============================] - 232s 5s/step - loss: 0.4951 - accuracy: 0.7801 - recall: 0.7429 - val_loss: 0.5061 - val_accuracy: 0.7690 - val_recall: 0.8129
Total Time Elapsed:  19  minutes  20  seconds


## 2x2x1x1C1D

Try deeper structure with another layer.

In [33]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 240s 6s/step - loss: 0.5649 - accuracy: 0.7266 - recall: 0.6895 - val_loss: 0.5360 - val_accuracy: 0.7690 - val_recall: 0.8810
Epoch 2/5
43/43 [==============================] - 254s 6s/step - loss: 0.5254 - accuracy: 0.7533 - recall: 0.7236 - val_loss: 0.4926 - val_accuracy: 0.7673 - val_recall: 0.7721
Epoch 3/5
43/43 [==============================] - 247s 6s/step - loss: 0.5106 - accuracy: 0.7645 - recall: 0.7281 - val_loss: 0.4943 - val_accuracy: 0.7808 - val_recall: 0.7483
Epoch 4/5
43/43 [==============================] - 245s 6s/step - loss: 0.5046 - accuracy: 0.7734 - recall: 0.7355 - val_loss: 0.4881 - val_accuracy: 0.7740 - val_recall: 0.7687
Epoch 5/5
43/43 [==============================] - 279s 6s/step - loss: 0.5013 - accuracy: 0.7682 - recall: 0.7370 - val_loss: 0.4898 - val_accuracy: 0.7825 - val_recall: 0.7687
Total Time Elapsed:  21  minutes  34  seconds


## 2x2x1x1C1D Adam

Try Adam optimizer instead of RMSprop for faster convergence.

In [34]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 267s 6s/step - loss: 0.6146 - accuracy: 0.6776 - recall: 0.6984 - val_loss: 0.4986 - val_accuracy: 0.7774 - val_recall: 0.7245
Epoch 2/5
43/43 [==============================] - 296s 7s/step - loss: 0.5243 - accuracy: 0.7675 - recall: 0.7103 - val_loss: 0.5341 - val_accuracy: 0.7454 - val_recall: 0.8129
Epoch 3/5
43/43 [==============================] - 307s 7s/step - loss: 0.5114 - accuracy: 0.7675 - recall: 0.7221 - val_loss: 0.5156 - val_accuracy: 0.7639 - val_recall: 0.8401
Epoch 4/5
43/43 [==============================] - 295s 7s/step - loss: 0.5015 - accuracy: 0.7741 - recall: 0.7207 - val_loss: 0.5066 - val_accuracy: 0.7605 - val_recall: 0.7959
Epoch 5/5
43/43 [==============================] - 307s 7s/step - loss: 0.5096 - accuracy: 0.7734 - recall: 0.7489 - val_loss: 0.5438 - val_accuracy: 0.7858 - val_recall: 0.7381
Total Time Elapsed:  25  minutes  6  seconds


## 2x2x1x1C1D Adam / BatchNorm

Add BatchNormalization for improved performance.

In [5]:
model = models.Sequential()
model.add(layers.Conv2D(96, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(96, 3, activation = 'relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, 3, activation = 'relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['accuracy'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 495s 12s/step - loss: 0.9597 - accuracy: 0.6976 - val_loss: 0.7722 - val_accuracy: 0.4958
Epoch 2/5
43/43 [==============================] - 493s 11s/step - loss: 0.5193 - accuracy: 0.7585 - val_loss: 0.8242 - val_accuracy: 0.4958
Epoch 3/5
43/43 [==============================] - 499s 12s/step - loss: 0.5028 - accuracy: 0.7682 - val_loss: 0.8062 - val_accuracy: 0.4958
Epoch 4/5
43/43 [==============================] - 521s 12s/step - loss: 0.4881 - accuracy: 0.7667 - val_loss: 0.9024 - val_accuracy: 0.4958
Epoch 5/5
43/43 [==============================] - 518s 12s/step - loss: 0.4793 - accuracy: 0.7853 - val_loss: 0.7885 - val_accuracy: 0.4958
Total Time Elapsed:  43  minutes  3  seconds


## 1x1C1D Adam/BatchNorm

Try simpler model with Adam optimizer and Batch Normalization layers.

In [6]:
model = models.Sequential()
model.add(layers.Conv2D(96, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['accuracy'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 160s 4s/step - loss: 9.4350 - accuracy: 0.6649 - val_loss: 27.0276 - val_accuracy: 0.4958
Epoch 2/5
43/43 [==============================] - 167s 4s/step - loss: 0.5592 - accuracy: 0.7013 - val_loss: 74.2437 - val_accuracy: 0.4958
Epoch 3/5
43/43 [==============================] - 165s 4s/step - loss: 0.5635 - accuracy: 0.7526 - val_loss: 119.3969 - val_accuracy: 0.4958
Epoch 4/5
43/43 [==============================] - 165s 4s/step - loss: 0.5146 - accuracy: 0.7749 - val_loss: 161.5224 - val_accuracy: 0.4958
Epoch 5/5
43/43 [==============================] - 167s 4s/step - loss: 0.4815 - accuracy: 0.7979 - val_loss: 200.8001 - val_accuracy: 0.4958
Total Time Elapsed:  14  minutes  5  seconds


## 1x1C1D BatchNorm

Try Batch Normalization with RMSprop optimizer.

In [7]:
model = models.Sequential()
model.add(layers.Conv2D(96, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(),
              metrics=['accuracy'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 174s 4s/step - loss: 38.4849 - accuracy: 0.6530 - val_loss: 37.3130 - val_accuracy: 0.4958
Epoch 2/5
43/43 [==============================] - 174s 4s/step - loss: 0.6234 - accuracy: 0.7288 - val_loss: 89.1527 - val_accuracy: 0.4958
Epoch 3/5
43/43 [==============================] - 177s 4s/step - loss: 1.0666 - accuracy: 0.7325 - val_loss: 125.1665 - val_accuracy: 0.4958
Epoch 4/5
43/43 [==============================] - 184s 4s/step - loss: 0.5839 - accuracy: 0.7467 - val_loss: 195.6121 - val_accuracy: 0.4958
Epoch 5/5
43/43 [==============================] - 188s 4s/step - loss: 0.5816 - accuracy: 0.7444 - val_loss: 284.6227 - val_accuracy: 0.4958
Total Time Elapsed:  15  minutes  19  seconds


## 4x1C1D

Try even deeper structure with 4 individual layers.

In [9]:
model = models.Sequential()
model.add(layers.Conv2D(96, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(),
              metrics=['accuracy'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 112s 3s/step - loss: 0.6164 - accuracy: 0.7073 - val_loss: 0.5438 - val_accuracy: 0.7538
Epoch 2/5
43/43 [==============================] - 132s 3s/step - loss: 0.5270 - accuracy: 0.7637 - val_loss: 0.5073 - val_accuracy: 0.7504
Epoch 3/5
43/43 [==============================] - 126s 3s/step - loss: 0.5187 - accuracy: 0.7652 - val_loss: 0.5017 - val_accuracy: 0.7926
Epoch 4/5
43/43 [==============================] - 131s 3s/step - loss: 0.5057 - accuracy: 0.7645 - val_loss: 0.4917 - val_accuracy: 0.7858
Epoch 5/5
43/43 [==============================] - 135s 3s/step - loss: 0.4820 - accuracy: 0.7786 - val_loss: 0.6690 - val_accuracy: 0.7302
Total Time Elapsed:  10  minutes  52  seconds


## 3x1C3D

Try deeper Dense layers.

In [6]:
model = models.Sequential()
model.add(layers.Conv2D(96, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(32, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(),
              metrics=['accuracy'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 109s 3s/step - loss: 0.6757 - accuracy: 0.7036 - val_loss: 0.5021 - val_accuracy: 0.7774
Epoch 2/5
43/43 [==============================] - 106s 2s/step - loss: 0.5202 - accuracy: 0.7571 - val_loss: 0.4945 - val_accuracy: 0.7673
Epoch 3/5
43/43 [==============================] - 108s 3s/step - loss: 0.5097 - accuracy: 0.7719 - val_loss: 0.5136 - val_accuracy: 0.7605
Epoch 4/5
43/43 [==============================] - 111s 3s/step - loss: 0.4750 - accuracy: 0.7831 - val_loss: 0.4928 - val_accuracy: 0.7673
Epoch 5/5
43/43 [==============================] - 111s 3s/step - loss: 0.4501 - accuracy: 0.7979 - val_loss: 0.4947 - val_accuracy: 0.7757
Total Time Elapsed:  9  minutes  20  seconds


## 2C3D

Try deeper Dense layers.

In [7]:
model = models.Sequential()
model.add(layers.Conv2D(96, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(32, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(),
              metrics=['accuracy'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 106s 2s/step - loss: 0.6112 - accuracy: 0.6947 - val_loss: 0.4981 - val_accuracy: 0.7673
Epoch 2/5
43/43 [==============================] - 112s 3s/step - loss: 0.5278 - accuracy: 0.7675 - val_loss: 0.5280 - val_accuracy: 0.7403
Epoch 3/5
43/43 [==============================] - 112s 3s/step - loss: 0.4761 - accuracy: 0.7816 - val_loss: 0.4874 - val_accuracy: 0.7707
Epoch 4/5
43/43 [==============================] - 110s 3s/step - loss: 0.4285 - accuracy: 0.8068 - val_loss: 0.5062 - val_accuracy: 0.7403
Epoch 5/5
43/43 [==============================] - 105s 2s/step - loss: 0.3692 - accuracy: 0.8366 - val_loss: 0.5771 - val_accuracy: 0.7538
Total Time Elapsed:  9  minutes  19  seconds


## 2x2C1D Dropout

Try adding Dropout to combat overfitting.

In [9]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(),
              metrics=['accuracy'])

start = timer()

history = model.fit(train_g, 
                    epochs=8, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/8
43/43 [==============================] - 258s 6s/step - loss: 0.7648 - accuracy: 0.6783 - val_loss: 0.5143 - val_accuracy: 0.7690
Epoch 2/8
43/43 [==============================] - 252s 6s/step - loss: 0.5347 - accuracy: 0.7533 - val_loss: 0.4884 - val_accuracy: 0.7690
Epoch 3/8
43/43 [==============================] - 281s 7s/step - loss: 0.5109 - accuracy: 0.7741 - val_loss: 0.4994 - val_accuracy: 0.7825
Epoch 4/8
43/43 [==============================] - 282s 7s/step - loss: 0.4900 - accuracy: 0.7808 - val_loss: 0.4836 - val_accuracy: 0.7808
Epoch 5/8
43/43 [==============================] - 296s 7s/step - loss: 0.4455 - accuracy: 0.7994 - val_loss: 0.5056 - val_accuracy: 0.7673
Epoch 6/8
43/43 [==============================] - 304s 7s/step - loss: 0.4215 - accuracy: 0.8105 - val_loss: 0.5104 - val_accuracy: 0.7723
Epoch 7/8
43/43 [==============================] - 293s 7s/step - loss: 0.3706 - accuracy: 0.8343 - val_loss: 0.5463 - val_accuracy: 0.7420
Epoch 8/8
43/43 [===

## 2x1x1C1D Dropout

Add Dropout to model to combat overfitting.

In [13]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=10, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/10
43/43 [==============================] - 285s 7s/step - loss: 0.6613 - accuracy: 0.6820 - recall: 0.6374 - val_loss: 0.5374 - val_accuracy: 0.7605 - val_recall: 0.7619
Epoch 2/10
43/43 [==============================] - 286s 7s/step - loss: 0.5611 - accuracy: 0.7370 - recall: 0.6999 - val_loss: 0.5480 - val_accuracy: 0.7622 - val_recall: 0.7483
Epoch 3/10
43/43 [==============================] - 288s 7s/step - loss: 0.5204 - accuracy: 0.7630 - recall: 0.6924 - val_loss: 0.5167 - val_accuracy: 0.7622 - val_recall: 0.7857
Epoch 4/10
43/43 [==============================] - 286s 7s/step - loss: 0.5299 - accuracy: 0.7645 - recall: 0.7221 - val_loss: 0.5492 - val_accuracy: 0.7673 - val_recall: 0.7823
Epoch 5/10
43/43 [==============================] - 256s 6s/step - loss: 0.5077 - accuracy: 0.7816 - recall: 0.7400 - val_loss: 0.5018 - val_accuracy: 0.7707 - val_recall: 0.7959
Epoch 6/10
43/43 [==============================] - 266s 6s/step - loss: 0.4859 - accuracy: 0.7779 - reca

## 2x1x1C1D Dropout Adam

Try Dropout and Adam optimizer.

In [4]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=10, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/10
43/43 [==============================] - 252s 6s/step - loss: 0.5885 - accuracy: 0.7229 - recall: 0.7296 - val_loss: 0.5153 - val_accuracy: 0.7673 - val_recall: 0.7279
Epoch 2/10
43/43 [==============================] - 254s 6s/step - loss: 0.5250 - accuracy: 0.7519 - recall: 0.7103 - val_loss: 0.5672 - val_accuracy: 0.7521 - val_recall: 0.8061
Epoch 3/10
43/43 [==============================] - 228s 5s/step - loss: 0.5034 - accuracy: 0.7689 - recall: 0.7266 - val_loss: 0.5486 - val_accuracy: 0.7757 - val_recall: 0.7653
Epoch 4/10
43/43 [==============================] - 228s 5s/step - loss: 0.5106 - accuracy: 0.7779 - recall: 0.7489 - val_loss: 0.5239 - val_accuracy: 0.7454 - val_recall: 0.8095
Epoch 5/10
43/43 [==============================] - 225s 5s/step - loss: 0.4840 - accuracy: 0.7779 - recall: 0.7370 - val_loss: 0.5196 - val_accuracy: 0.7369 - val_recall: 0.8401
Epoch 6/10
43/43 [==============================] - 223s 5s/step - loss: 0.4715 - accuracy: 0.7964 - reca

## 3C1D -- Fewer parameters

Tried fewer numbers of filters for Conv2D layers.

In [6]:
model = models.Sequential()
model.add(layers.Conv2D(32, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(32, 3, activation = 'relu'))
model.add(layers.Conv2D(32, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 141s 3s/step - loss: 0.7555 - accuracy: 0.6560 - recall: 0.6077 - val_loss: 0.5046 - val_accuracy: 0.7656 - val_recall: 0.7483
Epoch 2/5
43/43 [==============================] - 154s 4s/step - loss: 0.5190 - accuracy: 0.7585 - recall: 0.7013 - val_loss: 0.4900 - val_accuracy: 0.7690 - val_recall: 0.7619
Epoch 3/5
43/43 [==============================] - 142s 3s/step - loss: 0.4789 - accuracy: 0.7771 - recall: 0.7147 - val_loss: 0.5276 - val_accuracy: 0.7690 - val_recall: 0.8061
Epoch 4/5
43/43 [==============================] - 146s 3s/step - loss: 0.4424 - accuracy: 0.7883 - recall: 0.7652 - val_loss: 0.5293 - val_accuracy: 0.7555 - val_recall: 0.7993
Epoch 5/5
43/43 [==============================] - 142s 3s/step - loss: 0.3296 - accuracy: 0.8588 - recall: 0.8306 - val_loss: 0.8256 - val_accuracy: 0.7656 - val_recall: 0.7619
Total Time Elapsed:  12  minutes  20  seconds
